In [31]:
# Imports
import torch
from torch import nn
from src.utils.preprocess_utils import midi_to_note_indices, notes_to_midi
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [32]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x):
        # x: [batch_size, sequence_length]
        x = x.long()
        embedded = self.embedding(x)  # [batch_size, sequence_length, embedding_size]
        lstm_out, _ = self.lstm(embedded)  # [batch_size, sequence_length, hidden_size]
        output = self.fc(lstm_out)  # [batch_size, sequence_length, vocab_size]
        
        return output

In [33]:
# Load model
model_name = "2025-01-05_19-23-45_e64_h128_l5_sl50"
model = torch.load(f'weights/model_2/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_24388\3063146233.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_2/{model_name}')


LSTMModel(
  (embedding): Embedding(85, 64)
  (lstm): LSTM(64, 128, num_layers=5, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [34]:
# Load input
input = midi_to_note_indices("data/input.mid", 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.1
print(input)

[24, 26, 31, 36]


In [35]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_tensor = torch.tensor(start_sequence, dtype=torch.long).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(generated_sequence_tensor, dtype=torch.long).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 60, 56, 35, 19, 52, 55, 33, 48, 54, 38, 50, 41, 38, 59, 56, 55, 54, 52, 49, 45, 46, 51, 28, 33, 48, 46, 34, 42, 43, 39, 34, 42, 33, 46, 45, 51, 49, 34, 28, 46, 58, 33, 45, 41, 29, 33, 48, 45, 41, 29, 33, 48, 50, 29, 54, 50, 38, 41, 29, 59, 50, 49, 41, 38, 41, 29, 59, 50, 47, 29, 41, 38, 35, 29, 41, 38, 35, 29, 41, 38, 34, 46, 28, 37, 46, 43, 34, 28, 46, 43, 55, 52, 34, 28, 49, 37, 52, 28, 58, 46, 34, 28, 46, 58, 55, 43, 39, 34, 46, 28, 43, 45, 46, 48, 33, 30, 17, 17, 48, 45, 47, 47, 35, 47, 29, 47, 35, 47, 29, 47, 35, 38, 50, 47, 29, 59, 47, 45, 33, 30, 38, 35, 29, 17, 45, 33, 30, 38, 35, 47, 29, 59, 50, 47, 35, 59, 47, 29, 35, 53, 19, 50, 38, 35, 47, 29, 41, 59, 47, 35, 50, 38, 35, 47, 50, 48, 45, 33, 30, 48, 51, 39, 42, 33, 21, 48, 51, 39, 43, 19, 46, 43, 55, 52, 34, 37, 28, 51, 52, 34]


In [36]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_2/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_2/2025-01-05_19-29-54_t0.1_w200_M_2025-01-05_19-23-45_e64_h128_l5_sl50.mid
